# Graph Databases - Neo4j & Graph Traversal

Graph databases are designed to store, map, and query relationships between data. Unlike relational databases that use tables and joins, graph databases use **nodes**, **edges**, and **properties** to represent and store data.

## Why Graph Databases?

| Feature | Relational DB | Graph DB |
|---------|--------------|----------|
| Relationship queries | Expensive JOINs | Native traversal |
| Schema flexibility | Rigid schema | Schema-optional |
| Query complexity | Grows with depth | Constant time hops |
| Use case fit | Tabular data | Connected data |

## Property Graph Model

The **Property Graph Model** is the foundation of most graph databases including Neo4j.

### Core Components

```
┌─────────────────────────────────────────────────────────────┐
│                    PROPERTY GRAPH MODEL                      │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│   ┌──────────┐      RELATIONSHIP        ┌──────────┐        │
│   │   NODE   │ ───────────────────────► │   NODE   │        │
│   │  (Alice) │      [:FOLLOWS]          │   (Bob)  │        │
│   └──────────┘      since: 2020         └──────────┘        │
│       │                                      │              │
│       ▼                                      ▼              │
│   Properties:                           Properties:         │
│   - name: "Alice"                       - name: "Bob"       │
│   - age: 30                             - age: 25           │
│   - city: "NYC"                         - city: "LA"        │
│                                                              │
│   Labels: [:Person, :User]              Labels: [:Person]   │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### 1. Nodes (Vertices)
- Represent **entities** in your data model
- Can have one or more **labels** (like types/categories)
- Store data as **key-value properties**

### 2. Edges (Relationships)
- Connect nodes and represent **relationships**
- Always have a **direction** (from → to)
- Have exactly one **type** (e.g., FOLLOWS, PURCHASED)
- Can also store **properties**

### 3. Properties
- Key-value pairs stored on nodes or relationships
- Support various data types: strings, numbers, booleans, arrays

## Setup: NetworkX for Graph Operations

In [ ]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
from typing import List, Dict, Any

## Building a Property Graph with NetworkX

Let's create a social network graph to demonstrate graph concepts.

In [ ]:
# Create a directed graph (DiGraph) to support relationship directions
G = nx.DiGraph()

# Add nodes with properties (simulating Neo4j nodes with labels and properties)
users = [
    {"id": 1, "name": "Alice", "age": 30, "city": "NYC", "label": "Person"},
    {"id": 2, "name": "Bob", "age": 25, "city": "LA", "label": "Person"},
    {"id": 3, "name": "Charlie", "age": 35, "city": "Chicago", "label": "Person"},
    {"id": 4, "name": "Diana", "age": 28, "city": "NYC", "label": "Person"},
    {"id": 5, "name": "Eve", "age": 32, "city": "Boston", "label": "Person"},
]

for user in users:
    G.add_node(user["id"], **user)

# Add edges (relationships) with properties
relationships = [
    (1, 2, {"type": "FOLLOWS", "since": 2020}),
    (1, 3, {"type": "FOLLOWS", "since": 2019}),
    (2, 3, {"type": "FOLLOWS", "since": 2021}),
    (2, 4, {"type": "FRIENDS", "since": 2018}),
    (3, 5, {"type": "FOLLOWS", "since": 2020}),
    (4, 1, {"type": "FOLLOWS", "since": 2022}),
    (5, 1, {"type": "FRIENDS", "since": 2019}),
    (4, 5, {"type": "FOLLOWS", "since": 2021}),
]

for source, target, props in relationships:
    G.add_edge(source, target, **props)

print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
print(f"\nNodes: {list(G.nodes(data=True))}")

## Graph Traversal Operations

Graph traversal is the process of visiting nodes in a graph. Common operations include:
- **BFS (Breadth-First Search)**: Explore level by level
- **DFS (Depth-First Search)**: Explore as deep as possible first
- **Shortest Path**: Find the shortest route between nodes

In [ ]:
# Basic graph queries - simulating Neo4j operations

def find_node_by_property(graph: nx.DiGraph, key: str, value: Any) -> List[int]:
    """Find nodes matching a property value (like Neo4j MATCH ... WHERE)"""
    return [node for node, data in graph.nodes(data=True) if data.get(key) == value]

def get_outgoing_relationships(graph: nx.DiGraph, node_id: int) -> List[Dict]:
    """Get all outgoing relationships from a node"""
    return [
        {"from": node_id, "to": neighbor, **graph.edges[node_id, neighbor]}
        for neighbor in graph.successors(node_id)
    ]

def get_incoming_relationships(graph: nx.DiGraph, node_id: int) -> List[Dict]:
    """Get all incoming relationships to a node"""
    return [
        {"from": predecessor, "to": node_id, **graph.edges[predecessor, node_id]}
        for predecessor in graph.predecessors(node_id)
    ]

# Find all people in NYC
nyc_residents = find_node_by_property(G, "city", "NYC")
print("People in NYC:", [G.nodes[n]["name"] for n in nyc_residents])

# Who does Alice follow?
alice_follows = get_outgoing_relationships(G, 1)
print(f"\nAlice follows: {[G.nodes[r['to']]['name'] for r in alice_follows]}")

# Who follows Alice?
alice_followers = get_incoming_relationships(G, 1)
print(f"Alice's followers: {[G.nodes[r['from']]['name'] for r in alice_followers]}")

In [ ]:
# Advanced traversal: Find friends of friends (2-hop query)

def friends_of_friends(graph: nx.DiGraph, node_id: int, relationship_type: str = None) -> set:
    """Find nodes 2 hops away (friends of friends)"""
    direct_connections = set(graph.successors(node_id))
    fof = set()
    
    for friend in direct_connections:
        for fof_candidate in graph.successors(friend):
            if fof_candidate != node_id and fof_candidate not in direct_connections:
                if relationship_type is None or graph.edges[friend, fof_candidate].get("type") == relationship_type:
                    fof.add(fof_candidate)
    return fof

# Find friends-of-friends for Alice
alice_fof = friends_of_friends(G, 1)
print(f"Alice's friends of friends: {[G.nodes[n]['name'] for n in alice_fof]}")

# Shortest path between nodes
try:
    path = nx.shortest_path(G, source=1, target=5)
    path_names = [G.nodes[n]["name"] for n in path]
    print(f"\nShortest path from Alice to Eve: {' → '.join(path_names)}")
except nx.NetworkXNoPath:
    print("No path exists")

## Neo4j Cypher Query Language

**Cypher** is Neo4j's declarative query language. It uses ASCII-art patterns to describe graph patterns.

### Cypher Syntax Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                     CYPHER PATTERN SYNTAX                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  NODES:          (variable:Label {property: value})             │
│  Example:        (p:Person {name: "Alice"})                     │
│                                                                  │
│  RELATIONSHIPS:  -[variable:TYPE {props}]->                     │
│  Example:        -[:FOLLOWS {since: 2020}]->                    │
│                                                                  │
│  PATTERN:        (a)-[r]->(b)                                   │
│  Example:        (alice:Person)-[:FOLLOWS]->(bob:Person)        │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Cypher Query Examples (as strings - for reference when using Neo4j)

cypher_examples = {
    "CREATE_NODE": """
// Create a node with label and properties
CREATE (p:Person {name: 'Alice', age: 30, city: 'NYC'})
RETURN p
""",
    
    "CREATE_RELATIONSHIP": """
// Create a relationship between existing nodes
MATCH (a:Person {name: 'Alice'}), (b:Person {name: 'Bob'})
CREATE (a)-[:FOLLOWS {since: 2020}]->(b)
RETURN a, b
""",
    
    "MATCH_BASIC": """
// Find all Person nodes
MATCH (p:Person)
RETURN p.name, p.age
""",
    
    "MATCH_WITH_WHERE": """
// Find people in NYC over 25
MATCH (p:Person)
WHERE p.city = 'NYC' AND p.age > 25
RETURN p.name, p.age, p.city
""",
    
    "MATCH_RELATIONSHIP": """
// Find who Alice follows
MATCH (alice:Person {name: 'Alice'})-[:FOLLOWS]->(followed:Person)
RETURN followed.name AS following
""",
    
    "FRIENDS_OF_FRIENDS": """
// Find friends of friends (2-hop query)
MATCH (alice:Person {name: 'Alice'})-[:FOLLOWS]->()-[:FOLLOWS]->(fof:Person)
WHERE NOT (alice)-[:FOLLOWS]->(fof) AND alice <> fof
RETURN DISTINCT fof.name AS recommendation
""",
    
    "SHORTEST_PATH": """
// Find shortest path between two people
MATCH path = shortestPath(
    (alice:Person {name: 'Alice'})-[*]-(eve:Person {name: 'Eve'})
)
RETURN path, length(path) AS hops
""",
    
    "AGGREGATION": """
// Count followers per person
MATCH (p:Person)<-[:FOLLOWS]-(follower:Person)
RETURN p.name, COUNT(follower) AS follower_count
ORDER BY follower_count DESC
""",
    
    "UPDATE_NODE": """
// Update node properties
MATCH (p:Person {name: 'Alice'})
SET p.age = 31, p.verified = true
RETURN p
""",
    
    "DELETE_RELATIONSHIP": """
// Delete a relationship
MATCH (a:Person {name: 'Alice'})-[r:FOLLOWS]->(b:Person {name: 'Bob'})
DELETE r
"""
}

# Display examples
for name, query in cypher_examples.items():
    print(f"\n{'='*60}")
    print(f"📌 {name}")
    print(f"{'='*60}")
    print(query)

## Graph Visualization with Plotly

Let's visualize our social network graph using Plotly for interactive exploration.

In [ ]:
def visualize_graph(graph: nx.DiGraph, title: str = "Graph Visualization") -> go.Figure:
    """Create an interactive Plotly visualization of a NetworkX graph"""
    
    # Use spring layout for node positioning
    pos = nx.spring_layout(graph, k=2, iterations=50, seed=42)
    
    # Create edge traces
    edge_traces = []
    
    for edge in graph.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        
        # Color based on relationship type
        edge_type = edge[2].get("type", "UNKNOWN")
        color = "#1f77b4" if edge_type == "FOLLOWS" else "#2ca02c"
        
        edge_trace = go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode="lines",
            line=dict(width=2, color=color),
            hoverinfo="text",
            text=f"{graph.nodes[edge[0]]['name']} -{edge_type}-> {graph.nodes[edge[1]]['name']}",
            showlegend=False
        )
        edge_traces.append(edge_trace)
    
    # Create node trace
    node_x = [pos[node][0] for node in graph.nodes()]
    node_y = [pos[node][1] for node in graph.nodes()]
    
    # Node size based on degree (connections)
    node_sizes = [20 + 10 * graph.degree(node) for node in graph.nodes()]
    
    # Node colors based on city
    city_colors = {"NYC": "#e74c3c", "LA": "#3498db", "Chicago": "#2ecc71", 
                   "Boston": "#9b59b6", "Seattle": "#f39c12"}
    node_colors = [city_colors.get(graph.nodes[n].get("city", ""), "#95a5a6") for n in graph.nodes()]
    
    # Hover text with node properties
    hover_texts = [
        f"<b>{graph.nodes[n]['name']}</b><br>"
        f"Age: {graph.nodes[n]['age']}<br>"
        f"City: {graph.nodes[n]['city']}<br>"
        f"Connections: {graph.degree(n)}"
        for n in graph.nodes()
    ]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        marker=dict(
            size=node_sizes,
            color=node_colors,
            line=dict(width=2, color="white")
        ),
        text=[graph.nodes[n]["name"] for n in graph.nodes()],
        textposition="top center",
        hoverinfo="text",
        hovertext=hover_texts,
        showlegend=False
    )
    
    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])
    
    fig.update_layout(
        title=dict(text=title, font=dict(size=20)),
        showlegend=False,
        hovermode="closest",
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor="white",
        width=800,
        height=600,
        annotations=[
            dict(
                text="Node size = connections | Blue edges = FOLLOWS | Green edges = FRIENDS",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.5, y=-0.05,
                font=dict(size=12, color="gray")
            )
        ]
    )
    
    return fig

# Visualize our social network
fig = visualize_graph(G, "Social Network Graph")
fig

## Use Case 1: Social Network Analysis

Graph databases excel at social network analysis, enabling efficient queries for:
- Friend recommendations
- Influence detection
- Community discovery

In [ ]:
# Social Network Analysis: Find influencers and communities

# Calculate centrality metrics
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
pagerank = nx.pagerank(G)

# Create a summary DataFrame
analysis_df = pd.DataFrame([
    {
        "Person": G.nodes[n]["name"],
        "Degree Centrality": round(degree_centrality[n], 3),
        "Betweenness": round(betweenness_centrality[n], 3),
        "PageRank": round(pagerank[n], 3),
        "Followers": G.in_degree(n),
        "Following": G.out_degree(n)
    }
    for n in G.nodes()
])

print("📊 Social Network Analysis")
print("=" * 60)
print(analysis_df.sort_values("PageRank", ascending=False).to_string(index=False))

# Friend recommendation based on mutual connections
def recommend_friends(graph: nx.DiGraph, user_id: int, top_n: int = 3) -> List[Dict]:
    """Recommend friends based on mutual connections"""
    user_friends = set(graph.successors(user_id))
    candidates = {}
    
    for friend in user_friends:
        for fof in graph.successors(friend):
            if fof != user_id and fof not in user_friends:
                if fof not in candidates:
                    candidates[fof] = {"mutual": 0, "name": graph.nodes[fof]["name"]}
                candidates[fof]["mutual"] += 1
    
    sorted_candidates = sorted(candidates.items(), key=lambda x: x[1]["mutual"], reverse=True)
    return [{"id": c[0], **c[1]} for c in sorted_candidates[:top_n]]

print(f"\n\n🤝 Friend Recommendations for Alice:")
recommendations = recommend_friends(G, 1)
for rec in recommendations:
    print(f"  - {rec['name']} ({rec['mutual']} mutual connections)")

## Use Case 2: Fraud Detection

Graph databases are powerful for fraud detection by identifying:
- Circular transactions
- Account linking patterns
- Unusual relationship clusters

In [ ]:
# Fraud Detection Example: Identify suspicious transaction patterns

# Create a transaction graph
fraud_graph = nx.DiGraph()

# Add accounts
accounts = [
    {"id": "A001", "type": "checking", "owner": "John", "risk_score": 0.1},
    {"id": "A002", "type": "checking", "owner": "Jane", "risk_score": 0.2},
    {"id": "A003", "type": "checking", "owner": "Bob", "risk_score": 0.8},
    {"id": "A004", "type": "savings", "owner": "Shell Corp", "risk_score": 0.9},
    {"id": "A005", "type": "checking", "owner": "Alice", "risk_score": 0.15},
]

for acc in accounts:
    fraud_graph.add_node(acc["id"], **acc)

# Add transactions (some forming suspicious cycles)
transactions = [
    ("A001", "A002", {"amount": 1000, "type": "transfer"}),
    ("A002", "A003", {"amount": 950, "type": "transfer"}),
    ("A003", "A004", {"amount": 900, "type": "transfer"}),
    ("A004", "A001", {"amount": 850, "type": "transfer"}),  # Circular!
    ("A005", "A002", {"amount": 500, "type": "transfer"}),
    ("A003", "A001", {"amount": 200, "type": "transfer"}),  # Another cycle
]

for src, dst, props in transactions:
    fraud_graph.add_edge(src, dst, **props)

# Detect cycles (potential money laundering patterns)
def detect_cycles(graph: nx.DiGraph) -> List[List[str]]:
    """Detect all cycles in the transaction graph"""
    try:
        cycles = list(nx.simple_cycles(graph))
        return [c for c in cycles if len(c) >= 3]  # Only cycles with 3+ nodes
    except:
        return []

suspicious_cycles = detect_cycles(fraud_graph)

print("🚨 Fraud Detection Analysis")
print("=" * 60)
print(f"\nDetected {len(suspicious_cycles)} suspicious circular transaction patterns:")

for i, cycle in enumerate(suspicious_cycles, 1):
    cycle_path = " → ".join(cycle + [cycle[0]])  # Complete the cycle
    total_risk = sum(fraud_graph.nodes[n]["risk_score"] for n in cycle)
    print(f"\n  Pattern {i}: {cycle_path}")
    print(f"  Combined Risk Score: {total_risk:.2f}")
    print(f"  Accounts involved: {[fraud_graph.nodes[n]['owner'] for n in cycle]}")

In [ ]:
# Cypher query for fraud detection in Neo4j

fraud_cypher = """
// Detect circular money transfers (potential laundering)
MATCH path = (a:Account)-[:TRANSFER*3..6]->(a)
WHERE ALL(r IN relationships(path) WHERE r.amount > 100)
WITH path, 
     [n IN nodes(path) | n.risk_score] AS risks,
     [r IN relationships(path) | r.amount] AS amounts
WHERE reduce(total = 0.0, r IN risks | total + r) > 1.5
RETURN path,
       reduce(total = 0.0, r IN risks | total + r) AS total_risk,
       reduce(total = 0, a IN amounts | total + a) AS total_transferred
ORDER BY total_risk DESC
LIMIT 10
"""

print("📝 Neo4j Cypher Query for Fraud Detection:")
print(fraud_cypher)

## Use Case 3: Recommendation Engine

Graph databases power recommendation systems by leveraging:
- Collaborative filtering via graph traversal
- Content-based similarities
- Hybrid approaches combining both

In [ ]:
# Recommendation Engine: Product recommendations based on purchase patterns

rec_graph = nx.Graph()  # Undirected for similarity

# Add users and products
users_data = [
    {"id": "U1", "type": "user", "name": "Alice"},
    {"id": "U2", "type": "user", "name": "Bob"},
    {"id": "U3", "type": "user", "name": "Charlie"},
    {"id": "U4", "type": "user", "name": "Diana"},
]

products_data = [
    {"id": "P1", "type": "product", "name": "Python Book", "category": "books"},
    {"id": "P2", "type": "product", "name": "Data Science Course", "category": "courses"},
    {"id": "P3", "type": "product", "name": "Laptop Stand", "category": "accessories"},
    {"id": "P4", "type": "product", "name": "ML Book", "category": "books"},
    {"id": "P5", "type": "product", "name": "Keyboard", "category": "accessories"},
]

for item in users_data + products_data:
    rec_graph.add_node(item["id"], **item)

# Add purchases
purchases = [
    ("U1", "P1", {"rating": 5}),
    ("U1", "P2", {"rating": 4}),
    ("U2", "P1", {"rating": 5}),
    ("U2", "P4", {"rating": 4}),
    ("U3", "P2", {"rating": 5}),
    ("U3", "P3", {"rating": 3}),
    ("U4", "P3", {"rating": 4}),
    ("U4", "P5", {"rating": 5}),
]

for user, product, props in purchases:
    rec_graph.add_edge(user, product, **props)

def recommend_products(graph: nx.Graph, user_id: str, top_n: int = 3) -> List[Dict]:
    """Collaborative filtering: recommend products bought by similar users"""
    user_products = set(graph.neighbors(user_id))
    recommendations = {}
    
    # Find similar users (those who bought same products)
    for product in user_products:
        for other_user in graph.neighbors(product):
            if other_user != user_id and graph.nodes[other_user]["type"] == "user":
                # Check what else this similar user bought
                for other_product in graph.neighbors(other_user):
                    if other_product not in user_products:
                        if other_product not in recommendations:
                            recommendations[other_product] = {
                                "score": 0,
                                "name": graph.nodes[other_product]["name"],
                                "category": graph.nodes[other_product]["category"]
                            }
                        # Score based on rating and similarity
                        recommendations[other_product]["score"] += graph.edges[other_user, other_product]["rating"]
    
    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1]["score"], reverse=True)
    return [{"id": r[0], **r[1]} for r in sorted_recs[:top_n]]

# Get recommendations for Alice
print("🛍️ Product Recommendations")
print("=" * 60)

alice_recs = recommend_products(rec_graph, "U1")
print(f"\nRecommendations for Alice (based on similar users):")
for rec in alice_recs:
    print(f"  📦 {rec['name']} (Category: {rec['category']}, Score: {rec['score']})")

In [ ]:
# Neo4j Cypher for recommendations

recommendation_cypher = """
// Collaborative filtering recommendation query
MATCH (user:User {name: 'Alice'})-[:PURCHASED]->(product:Product)
      <-[:PURCHASED]-(similar_user:User)-[:PURCHASED]->(rec:Product)
WHERE NOT (user)-[:PURCHASED]->(rec)
  AND user <> similar_user
WITH rec, 
     COUNT(DISTINCT similar_user) AS shared_purchasers,
     AVG(similar_user.rating) AS avg_rating
RETURN rec.name AS product,
       rec.category AS category,
       shared_purchasers,
       avg_rating
ORDER BY shared_purchasers DESC, avg_rating DESC
LIMIT 5
"""

print("📝 Neo4j Cypher Query for Product Recommendations:")
print(recommendation_cypher)

## Graph Algorithm Analysis

Common graph algorithms used in graph databases:

In [ ]:
# Demonstrate common graph algorithms

print("📊 Graph Algorithm Analysis")
print("=" * 60)

# 1. Connected Components (for undirected graphs)
undirected_G = G.to_undirected()
components = list(nx.connected_components(undirected_G))
print(f"\n1. Connected Components: {len(components)}")
for i, comp in enumerate(components, 1):
    names = [G.nodes[n]["name"] for n in comp]
    print(f"   Component {i}: {names}")

# 2. Clustering Coefficient
clustering = nx.clustering(undirected_G)
print(f"\n2. Clustering Coefficients:")
for node, coef in clustering.items():
    print(f"   {G.nodes[node]['name']}: {coef:.3f}")

# 3. Graph Density
density = nx.density(G)
print(f"\n3. Graph Density: {density:.3f}")
print(f"   (0 = no edges, 1 = fully connected)")

# 4. Diameter (longest shortest path)
if nx.is_strongly_connected(G):
    diameter = nx.diameter(G)
    print(f"\n4. Graph Diameter: {diameter} hops")
else:
    print(f"\n4. Graph is not strongly connected")
    # Calculate for largest strongly connected component
    largest_scc = max(nx.strongly_connected_components(G), key=len)
    subgraph = G.subgraph(largest_scc)
    print(f"   Largest SCC diameter: {nx.diameter(subgraph)} hops")

## Neo4j vs Other Graph Databases

| Feature | Neo4j | Amazon Neptune | ArangoDB | JanusGraph |
|---------|-------|---------------|----------|------------|
| **Query Language** | Cypher | Gremlin, SPARQL | AQL | Gremlin |
| **Graph Model** | Property Graph | Property Graph + RDF | Multi-model | Property Graph |
| **Scaling** | Horizontal (Enterprise) | Managed, auto-scaling | Horizontal | Horizontal |
| **ACID Compliance** | Yes | Yes | Yes | Configurable |
| **Best For** | General purpose | AWS ecosystem | Multi-model needs | Big data graphs |
| **Learning Curve** | Low (Cypher is intuitive) | Medium | Medium | High |

## 📌 Key Takeaways

### When to Use Graph Databases

✅ **Use Graph Databases When:**
- Data has complex, many-to-many relationships
- You need to traverse relationships efficiently (social networks, recommendations)
- Query depth is unknown or variable (fraud detection, path finding)
- Schema flexibility is important (evolving data models)
- Real-time recommendations or analytics are required

❌ **Avoid Graph Databases When:**
- Data is primarily tabular with simple relationships
- Bulk aggregations are the primary use case
- Simple CRUD operations dominate
- Team lacks graph modeling expertise

### Property Graph Model Essentials

```
NODES       → Entities (People, Products, Accounts)
EDGES       → Relationships (FOLLOWS, PURCHASED, TRANSFERRED)
PROPERTIES  → Attributes on both (name, timestamp, amount)
LABELS      → Categories/Types for nodes (Person, Product)
```

### Core Cypher Patterns

```cypher
// Create
CREATE (n:Label {prop: value})

// Read
MATCH (n:Label)-[r:REL]->(m) WHERE n.prop = value RETURN n, r, m

// Update
MATCH (n:Label) SET n.prop = newValue

// Delete
MATCH (n:Label) DETACH DELETE n
```

### Common Use Cases

| Use Case | Graph Advantage |
|----------|----------------|
| **Social Networks** | Native relationship traversal |
| **Fraud Detection** | Pattern matching, cycle detection |
| **Recommendations** | Collaborative filtering via paths |
| **Knowledge Graphs** | Semantic relationships, inference |
| **Network/IT Ops** | Dependency mapping, impact analysis |
| **Supply Chain** | Path optimization, bottleneck detection |